# Freezing mode on a high symmetry structure
## prepared by Kishwar-E Hasin

Steps:
1. Create matrix with direct position of HS POSCAR
2. Create lattice parameter matrix and then create lattive vector with diagonal elements 
3. Create HS_A matrix: multiplying HS columns by corresponding lv elements
4. Create sym mode matrix, e
5. Multiply by the array (sqrt2, sqrt2, 1)---(necessary when z element present with x or y)
6. Normalize the e matrix
7. Create the distortion matrix,  dist=Q_atom . normalized_e
8. Create LS_A matrix: HS_A + dist
9. Create LS direct position matrix with direct position : dividing LSA columns by corresponding lp elements
10. Write the LS matrix on a file


In [4]:
import pandas as pd
import numpy as np
import math
from itertools import islice


high_symmetry_file_name = "HS"
matrix_file_name = "mode_lsno_48_atom.xlsx"
cell_correction = np.array([np.sqrt(2.), np.sqrt(2.), 1])   # change according to your computational unit cell
#==========================================================

with open(high_symmetry_file_name) as lines:          
        lattice_parameter = np.genfromtxt(islice(lines, 2, 5))                ## Extract lattice parameter     
        lattice_vector = np.diag(lattice_parameter)
        for i, line in enumerate(lines):                                      ## Count the total number of atoms
            if i == 1:  
                n_atom = sum(map(int, line.strip().split()))      
                break
        high_symmetry = np.genfromtxt(islice(lines, 1, 1 + n_atom))           ## Extract positions from high symmetry POSCAR    
        high_symmetry_angstrom = high_symmetry * lattice_vector               ## High symmetry positions in Angstrom

        
### Define function to freeze mode of specified amplitude        
def freeze_mode(sheet_name, element_values, mode_name, Q):   
    irrep_data = pd.read_excel(matrix_file_name, sheet_name, skiprows=1)      
    
    displacement = []
    for element_name, value in element_values.items():
        df_element = pd.DataFrame(irrep_data[0:n_atom], columns=[f"x_{element_name}", f"y_{element_name}", f"z_{element_name}"])        
        z_nonzero = np.any(df_element[f"z_{element_name}"] != 0)
        xy_nonzero = np.any((df_element[f"x_{element_name}"] != 0) | (df_element[f"y_{element_name}"] != 0))   
        e_element_matrix = np.array(df_element) 
        if z_nonzero and xy_nonzero:
            e_element_matrix = np.array(df_element) * cell_correction
        
        e_element = e_element_matrix / (np.linalg.norm(e_element_matrix))
        displacement.append(value * e_element)

    sum_displacement = np.sum(displacement, axis=0)
    distortion = Q * sum_displacement
    return distortion
        
        
        
# ### Define function to freeze mode of specified amplitude        
# def freeze_mode(sheet_name, element_values, mode_name, Q):   
#     mode_data = pd.read_excel(matrix_file_name, sheet_name, skiprows=1)      
    
#     displacement = []
#     for element, value in element_values.items():
#         df_element = pd.DataFrame(mode_data[0:n_atom], columns=[f"x_{element}", f"y_{element}", f"z_{element}"])        
#         if np.any(df_element[f"z_{element}"] != 0) and (np.any(df_element[f"x_{element}"] != 0) or np.any(df_element[f"y_{element}"] != 0)):
#             e_element_matrix = np.array(e_element_d) * cell_correction)
#         else:
#             e_element_matrix = np.array(df_element)                      # Convert the irrep data to matrix  
#         e_element = e_element_matrix / (np.linalg.norm(e_element_matrix))
#         displacement.append(value * e_element)

#     sum_displacement = np.sum(displacement, axis=0)
#     distortion = Q * sum_displacement
#     return distortion



####=============== This portion should be modified according to freezing criteria =============
### Define values for each element in a dictionary for different modes of Gm5-
element_values_gm5 = {
    "A": 0.03584420034188983,
    "A'": -0.12906948571551885,
    "B": -0.17366852768585422,
    "O1": 0.08845470528253055,
    "O2B2": 0.030024750236645064,
    "O2B1": 0.056511020393716194,
    "O3": 0.08000754337417192
}

# Define values for other modes
element_values_x2 = {
    "A" : 0.06870390964725873,
    "O2B2" : -0.011459716066796577,
    "O2B1" : -0.18098377853101305
    # Define values for mode m2
}


element_values_x3 = {
"A" : 0.6010912948520581,
"A'" : -0.058562040817392136,
"B" : -0.06156498196970611,
"O1" : 0.5193874780653167,
"O2" : 0.7831729655236858,
"O3" : -0.5105345338386122
    # Define values for mode m3
}


# ### Define values for each element in a dictionary for different modes of M5-
# element_values_m5 = {
#     "A" : 0.5, #0.07184458023093804,
#     "A'" : 0.19782721950267215,
#     "B" : -0.26922087886863005,
#     "O1" : 0.030850770736436994,
#     "O2B2" : 0.05452636035610116,
#     "O2B1" : -0.021140927185505115,
#     "O3" : -0.008096457651749094
#     # Define values for mode m5
# }

# # Define values for other modes
# element_values_x2 = {
#     "A" : -0.011138785708579268,
#     "O2B2" : 0.0126596579962776,
#     "O2B1" : -0.20022064796097339
#     # Define values for mode m2
# }


# element_values_x3 = {
# "A" : 0.5990030544971396,
# "A'" : -0.0581901035847965,
# "B" : -0.06063202997981847,
# "O1" : 0.5190936248360177,
# "O2" : 0.7825550472080292,
# "O3" : -0.5077589849582294
#     # Define values for mode m3
# }


### Define Q for modes. Put 0 if you dont want to 
Q_m5 = 0 
Q_gm5 = 3.880024629 * 0.25773032279049285
Q_x2 = 5.156648161 * 0.19392441925789816
Q_x3 = 0.8131695919 * 1.2297557728738562 

# List of mode information tuples: (sheeet name, element value, mode name, amplitude factor Q)
modes = [
    ('gm5-(a-a)', element_values_gm5, "gm5", Q_gm5),
    ('x2+(0a)', element_values_x2, "x2", Q_x2),
    ('x3-(0a)', element_values_x3, "x3", Q_x3)
    #('x3-(a0)', element_values_x3, "x3", Q_x3)     #for Pnma
    #('m5-(0a)', element_values_m5, "m5", Q_m5)     #for Pnma
]
#================= Till above this line should be modified according to freezing criteria====================================================================

# Initialize total distortions to a zero array with the same shape as the first distortion matrix
total_distortions = np.zeros_like(freeze_mode(*modes[0]))

# Calculate and sum distortions
for sheet_name, element_values, mode_name, Q in modes:
    distortion = freeze_mode(sheet_name, element_values, mode_name, Q)
    #print(f"Distortion for {mode_name}:", distortion)
    total_distortions += distortion
#print("Total distortions:", total_distortions)    

low_symmetry_angstrom = high_symmetry_angstrom + total_distortions     # Calculate lsa by adding total_distortions to hsa
low_symmetry_positions = low_symmetry_angstrom / lattice_vector        # Divide lsa by np.diag(lp) to calculate ls

### Write the low symmetry POSCAR in a file ####
with open(high_symmetry_file_name) as f:
    data_list = f.readlines()
    data_list[0] = "new_poscar\n"
    del data_list[8:]
    mat = np.matrix(low_symmetry_positions)
    with open('POSCAR', 'w') as f:
        f.writelines(data_list)
        np.savetxt(f, mat, fmt='  %.16f', delimiter='     ')

In [2]:
import pandas as pd
import numpy as np

# Constants
# Calculate sum_all_distortions
element_values_m5 = {
    "A" : 0.07184458023093804,
    "A'" : 0.19782721950267215,
    "B" : -0.26922087886863005,
    "O1" : 0.030850770736436994,
    "O2B2" : 0.05452636035610116,
    "O2B1" : -0.021140927185505115,
    "O3" : -0.008096457651749094
    # Define values for mode m5
}

# Define values for other modes
element_values_x2 = {
    "A" : -0.011138785708579268,
    "O2B2" : 0.0126596579962776,
    "O2B1" : -0.20022064796097339
    # Define values for mode m2
}


element_values_x3 = {
"A" : 0.5990030544971396,
"A'" : -0.0581901035847965,
"B" : -0.06063202997981847,
"O1" : 0.5190936248360177,
"O2" : 0.7825550472080292,
"O3" : -0.5077589849582294
    # Define values for mode m3
}
Q_m5, Q_gm5, Q_x2, Q_x3 = 1, 0, 1, 1


HIGH_SYMMETRY_FILE_NAME = "HS"
MATRIX_FILE_NAME = "mode_rp_48.xlsx"

# Function to read lattice parameter and high symmetry positions
def read_high_symmetry_data(file_name):
    with open(high_symmetry_file_name) as lines:          
        lp = np.genfromtxt(islice(lines, 2, 5))                               ## Extract lattice parameter     
        
        for i, line in enumerate(lines):                                      ## Count the total number of atoms
            if i == 1:  
                n_atom = sum(map(int, line.strip().split()))      
                break
        hs = np.genfromtxt(islice(lines, 1, 1 + n_atom))                      ## Extract positions from high symmetry POSCAR    
        hsa = hs * np.diag(lp)                                                ## High symmetry positions in Angstrom
    
    return lp, n_atom, hs, hsa



# Function to freeze mode of specified amplitude
def freeze_mode(mode_data, n_atom, element_values, Q):
    displacement = []
    for element, value in element_values.items():
        df_element = pd.DataFrame(mode_data[:n_atom], columns=[f"x_{element}", f"y_{element}", f"z_{element}"])
        z_non_zero = np.any(df_element[f"z_{element}"] != 0)
        xy_non_zero = np.any((df_element[f"x_{element}"] != 0) | (df_element[f"y_{element}"] != 0))
        
        if z_non_zero and xy_non_zero:
            e_element_matrix = np.array(df_element) * np.array([np.sqrt(2.), np.sqrt(2.), 1])
        else:
            e_element_matrix = np.array(df_element)
        
        e_element = e_element_matrix / (np.linalg.norm(e_element_matrix))
        displacement.append(value * e_element)

    sum_displacement = np.sum(displacement, axis=0)
    distortion = Q * sum_displacement
    return distortion


# Write the low symmetry POSCAR in a file
def write_low_symmetry_poscar(file_name, data_list, ls):
    data_list[0] = "space group\n"
    del data_list[8:]
    mat = np.matrix(ls)
    with open(file_name, 'w') as f:
        f.writelines(data_list)
        np.savetxt(f, mat, fmt='  %.16f', delimiter='     ')


# Main Execution
if __name__ == "__main__":
    lp, n_atom, hs, hsa = read_high_symmetry_data(HIGH_SYMMETRY_FILE_NAME)
    distortion_x2 = freeze_mode('x2+(0a)', n_atom, element_values_x2, Q_x2)
    distortion_x3 = freeze_mode('x3-(a0)', n_atom, element_values_x3, Q_x3)
    distortion_m5 = freeze_mode('m5-(0a)', n_atom, element_values_m5, Q_m5)

    sum_all_distortions = distortion_x2 + distortion_x3 + distortion_m5

    # Write low symmetry POSCAR
    with open(HIGH_SYMMETRY_FILE_NAME) as f:
        data_list = f.readlines()
        write_low_symmetry_poscar('POSCAR', data_list, hsa + sum_all_distortions)
    
        

ValueError: DataFrame constructor not properly called!